# Reprojection template

Use this when you want to change a series of images so that they are aligned with one of the imags.

In [3]:
from pathlib import Path

import numpy as np

from astropy.nddata import CCDData
from ccdproc import ImageFileCollection
from reproject import reproject_interp
from stellarphot.settings.fits_opener import FitsOpener

## Change the values here

In [8]:
image_folder = '.'  # '.' means the same folder as the notebook.

ipath = Path(image_folder)

# You can call this anything you want.
folder_for_shifted_images = 'shifted'

# Subract median background value?
subtract_median = True

### Choose your "reference" image

In [ ]:
fo = FitsOpener()
fo.file_chooser

In [ ]:
# This can be any one of the images. If one image is offset from all
# of the others do not choose that image.
reference_image = fo.file_chooser.value #'kelt-16-b-S001-R001-C004-r.fit'
print(reference_image)

### Run the rest but you should not need to modify anything

In [ ]:
p = Path(folder_for_shifted_images)
p.mkdir(exist_ok=True)
ccd_ref = CCDData.read(ipath / Path(reference_image).name)
wcs_ref = ccd_ref.wcs

### This cell does this shifting. It will update as it completes images.

In [ ]:
ifc = ImageFileCollection(image_folder)

n_images = len(ifc.summary) + 1
i = 0

for ccd, fname in ifc.ccds(return_fname=True):
    print(f'\n{i} done 🎉, {n_images - i - 1} left\n')
    i = i + 1
    print(f'\tStarting to shift image {fname}...')
    outfile = p / fname
    if outfile.exists():
        print(f'\t\t...skipping because it is already shifted')
        continue

    if subtract_median:
        median = np.nanmedian(ccd.data)
    else:
        median = 0

    new_data, footprint = reproject_interp(ccd, wcs_ref,
                                           shape_out=ccd_ref.shape, order=1)
    ccd.data = new_data - median
    ccd.wcs = wcs_ref
    ccd.write(p / fname)
    print(f'\t\t...Finished shifting image')

print('\n🎉 All done ✨')